In [24]:
import pandas as pd

file_path = '../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target.txt'

In [25]:
#  read the file
trace = pd.read_csv(file_path, sep=' ')
trace.columns = ['ByteOffset', 'IOSize']
print(len(trace))
print(trace.head())

5899528
   ByteOffset  IOSize
0  3201773568    8192
1  3155169280    4096
2  3155156992    4096
3  3161305088    8192
4  3154132992    4096


In [27]:
# declare dataframe variable to store converted trace
converted_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])
counter = 0
# READ each line of the trace
for index, row in trace.iterrows():
    converted_list = []
    number_of_lba = row['IOSize'] / 8
    for i in range(int(number_of_lba)):
        converted_list.append({'ByteOffset': row['ByteOffset'] + i * 8, 'IOSize': 8})
        counter += 1
        if counter % 10000 == 0:
            print(counter)
    # converted_list to dataframe and concatenate to converted_trace
    converted_trace = pd.concat([converted_trace, pd.DataFrame(converted_list)], ignore_index=True)
    if counter > 1576129:
        break
    
converted_trace.columns = ['ByteOffset', 'IOSize']
print(len(converted_trace))


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [28]:
# Save the converted trace
converted_trace.to_csv('../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target-converted.txt', sep=' ', index=False)